In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/Colab\ Notebooks/rag_training

/content/drive/MyDrive/Colab Notebooks/rag_training


In [3]:
%ls

dummy_rag/  rag_advanced/       rag_basic.ipynb  rag_vectordb.ipynb
milvus.db   rag_advanced.ipynb  rag_vectordb/


In [3]:
%%bash

uv pip install haystack-ai
uv pip install milvus_haystack
uv pip install pymilvus
uv pip install python-docx
uv pip install docling-haystack

Using Python 3.11.13 environment at: /usr
Resolved 40 packages in 740ms
Prepared 6 packages in 60ms
Installed 6 packages in 13ms
 + backoff==2.2.1
 + filetype==1.2.0
 + haystack-ai==2.15.2
 + haystack-experimental==0.12.0
 + lazy-imports==1.0.0
 + posthog==6.1.1
Using Python 3.11.13 environment at: /usr
Resolved 51 packages in 404ms
 Downloaded grpcio
 Downloaded milvus-lite
Prepared 6 packages in 818ms
Uninstalled 1 package in 6ms
Installed 6 packages in 6ms
 - grpcio==1.73.1
 + grpcio==1.67.1
 + milvus-haystack==0.0.16
 + milvus-lite==2.5.1
 + pymilvus==2.5.13
 + python-dotenv==1.1.1
 + ujson==5.10.0
Using Python 3.11.13 environment at: /usr
Audited 1 package in 73ms
Using Python 3.11.13 environment at: /usr
Resolved 3 packages in 49ms
Prepared 1 package in 24ms
Installed 1 package in 2ms
 + python-docx==1.2.0
Using Python 3.11.13 environment at: /usr
Resolved 119 packages in 1.86s
   Building pylatexenc==2.10
 Downloaded pypdfium2
 Downloaded easyocr
      Built pylatexenc==2.10
 Do

In [4]:
from pathlib import Path
import glob
import os
from typing import List
from haystack import Pipeline
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import OpenAIDocumentEmbedder, OpenAITextEmbedder
from getpass import getpass
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from docling_haystack.converter import DoclingConverter
from docling.chunking import HybridChunker
from haystack.components.rankers import SentenceTransformersSimilarityRanker
from haystack.components.joiners import ListJoiner
from haystack.components.builders import (
    AnswerBuilder,
    ChatPromptBuilder,
    PromptBuilder,
)
from haystack.components.converters import OutputAdapter
from haystack.components.generators import OpenAIGenerator
from milvus_haystack import MilvusDocumentStore, MilvusEmbeddingRetriever
from haystack_experimental.components.retrievers import ChatMessageRetriever
from haystack_experimental.components.writers import ChatMessageWriter
from haystack_experimental.chat_message_stores.in_memory import InMemoryChatMessageStore

In [5]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

Enter OpenAI API key:··········


In [6]:
DOCUMENTS_DIR = Path("./dummy_rag/documents_dir")
FILES = [file.resolve() for file in DOCUMENTS_DIR.rglob("*") if file.is_file()]

In [7]:
EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
converter = DoclingConverter(chunker=HybridChunker(tokenizer=EMBED_MODEL_ID))
documents = converter.run(paths=["https://arxiv.org/pdf/2408.09869"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2938 > 512). Running this sequence through the model will result in indexing errors


In [8]:
CHUNK_ID = 2
print(f'Chunk Heading: {documents["documents"][CHUNK_ID].meta["dl_meta"]["meta"]["headings"]}\n')
print("Chunk content:")
print(f'{documents["documents"][CHUNK_ID].content}\n')
doc_items = documents["documents"][CHUNK_ID].meta["dl_meta"]["meta"]["doc_items"][0]
print("Document Items:")
for k, v in doc_items.items():
    if k == 'prov':
      print(v[0])
    else:
      print(f"{k}: {v}")

Chunk Heading: ['1 Introduction']

Chunk content:
1 Introduction
Converting PDF documents back into a machine-processable format has been a major challenge for decades due to their huge variability in formats, weak standardization and printing-optimized characteristic, which discards most structural features and metadata. With the advent of LLMs and popular application patterns such as retrieval-augmented generation (RAG), leveraging the rich content embedded in PDFs has become ever more relevant. In the past decade, several powerful document understanding solutions have emerged on the market, most of which are commercial software, cloud offerings [3] and most recently, multi-modal vision-language models. As of today, only a handful of open-source tools cover PDF conversion, leaving a significant feature and quality gap to proprietary solutions.
With Docling , we open-source a very capable and efficient document conversion tool which builds on the powerful, specialized AI models and da

In [9]:
connection_args={"uri": "./rag_advanced/milvus.db"}
document_store = MilvusDocumentStore(
    connection_args=connection_args,
    drop_old=True,
)

In [10]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_component(
                    "converter",
                    DoclingConverter(chunker=HybridChunker(tokenizer=EMBED_MODEL_ID))
                )
indexing_pipeline.add_component("embedder", OpenAIDocumentEmbedder())
indexing_pipeline.add_component("writer", DocumentWriter(document_store))
indexing_pipeline.connect("converter", "embedder")
indexing_pipeline.connect("embedder", "writer")
indexing_pipeline.run({"converter": {"paths": FILES}})

Calculating embeddings: 2it [00:03,  1.79s/it]


{'embedder': {'meta': {'model': 'text-embedding-ada-002-v2',
   'usage': {'prompt_tokens': 11054, 'total_tokens': 11054}}},
 'writer': {'documents_written': 57}}

In [11]:
generation_kwargs = {"max_completion_tokens":32768,
                    "temperature":0.5,
                    "seed": 42,}

ranker_query_prefix = "Given the following user query, determine if the information provided below addresses the inquiry accurately."
ranker = SentenceTransformersSimilarityRanker(query_prefix=ranker_query_prefix, top_k=5)
ranker.warm_up()

In [12]:
with open("./dummy_rag/RAG_TEMPLATE.txt") as rag_file:
    rag_template = rag_file.read()

with open("./dummy_rag/QUERY_REPHRASE_TEMPLATE.txt") as query_rephrase_file:
    query_rephrase_template = query_rephrase_file.read()

In [13]:
system_message = ChatMessage.from_system("You are a helpful AI assistant using provided supporting documents and conversation history to assist humans")
user_prompt_template = ChatMessage.from_user(rag_template)

In [14]:
rag_pipeline = Pipeline()
memory_store = InMemoryChatMessageStore()

# components for query rephrasing
rag_pipeline.add_component("query_rephrase_prompt_builder", PromptBuilder(query_rephrase_template))
rag_pipeline.add_component("query_rephrase_llm", OpenAIGenerator())
rag_pipeline.add_component("list_to_str_adapter", OutputAdapter(template="{{ replies[0] }}", output_type=str))
rag_pipeline.add_component("text_embedder", OpenAITextEmbedder())
# components for RAG
rag_pipeline.add_component("retriever", MilvusEmbeddingRetriever(document_store=document_store, top_k=5))
rag_pipeline.add_component(instance=ranker, name="ranker")
rag_pipeline.add_component("prompt_builder", ChatPromptBuilder(variables=["query", "documents", "memories"], required_variables=["query", "documents", "memories"]))
rag_pipeline.add_component("llm", OpenAIChatGenerator())

# components for memory
rag_pipeline.add_component("memory_retriever", ChatMessageRetriever(memory_store))
rag_pipeline.add_component("memory_writer", ChatMessageWriter(memory_store))
rag_pipeline.add_component("memory_joiner", ListJoiner(List[ChatMessage]))


# connections for query rephrasing
rag_pipeline.connect("memory_retriever", "query_rephrase_prompt_builder.memories")
rag_pipeline.connect("query_rephrase_prompt_builder.prompt", "query_rephrase_llm")
rag_pipeline.connect("query_rephrase_llm.replies", "list_to_str_adapter.replies")
rag_pipeline.connect("list_to_str_adapter.output", "text_embedder.text")
rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")

# connections for RAG
rag_pipeline.connect("retriever.documents", "ranker.documents")
rag_pipeline.connect("ranker.documents", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder.prompt", "llm.messages")
rag_pipeline.connect("llm.replies", "memory_joiner")

# connections for memory
rag_pipeline.connect("memory_joiner", "memory_writer")
rag_pipeline.connect("memory_retriever", "prompt_builder.memories")

🚅 Components
  - query_rephrase_prompt_builder: PromptBuilder
  - query_rephrase_llm: OpenAIGenerator
  - list_to_str_adapter: OutputAdapter
  - text_embedder: OpenAITextEmbedder
  - retriever: MilvusEmbeddingRetriever
  - ranker: SentenceTransformersSimilarityRanker
  - prompt_builder: ChatPromptBuilder
  - llm: OpenAIChatGenerator
  - memory_retriever: ChatMessageRetriever
  - memory_writer: ChatMessageWriter
  - memory_joiner: ListJoiner
🛤️ Connections
  - query_rephrase_prompt_builder.prompt -> query_rephrase_llm.prompt (str)
  - query_rephrase_llm.replies -> list_to_str_adapter.replies (List[str])
  - list_to_str_adapter.output -> text_embedder.text (str)
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> ranker.documents (List[Document])
  - ranker.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])
  - llm.replies -> memory_joiner.values (List[ChatMessage])
  - memory_

In [15]:
messages = [system_message, user_prompt_template]
question = "Tell me a bit about the Quantum Stream product."

res = rag_pipeline.run(data={"query_rephrase_prompt_builder": {"query": question},
                          "prompt_builder": {"template": messages, "query": question},
                          "ranker": {"query": question},
                          "memory_joiner": {"values": [ChatMessage.from_user(question)]}},
                        include_outputs_from=["llm","query_rephrase_llm"])
search_query = res['query_rephrase_llm']['replies'][0]
print(f"🔎 Search Query: {search_query}")
assistant_resp = res['llm']['replies'][0]
print(f"🤖 {assistant_resp.text}")

   🔎 Search Query: Tell me a bit about the Quantum Stream product.
🤖 QuantumStream is a comprehensive platform designed for managing streaming data pipelines with a focus on security, compliance, and resilience. Its core features include robust data encryption capabilities to ensure data security as it flows through various systems [QuantumStream Data Encryption.docx, Chapter: NA, Page: NA]. 

The platform also provides a command-line interface (CLI) that allows developers, system administrators, and data engineers to manage these data streams efficiently. The CLI supports initialization of projects, configuration of stream parameters, and monitoring of real-time metrics [QuantumStream CLI Usage.docx, Chapter: NA, Page: NA]. 

Additionally, QuantumStream supports plugin architecture, allowing for customization to meet specific needs, although plugins must be written in Python and cannot be hot-reloaded currently [QuantumStream Plugin System.docx, Chapter: NA, Page: NA]. The API is desi